In [24]:
import os
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import tqdm
import matplotlib.pyplot as plt
from collections import Counter
import pickle
import re
import copy
import time

# Scoring
from sklearn.metrics import classification_report, f1_score
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device => ",device, ' torch ', torch.__version__)


# hyper parameters
SEED = 42
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    
#@title Hyper Parameters { display-mode: "both" }

EPOCHS             = 20
MAX_NO_OF_SPEAKERS = 232
MAX_DIALOGUE_LEN   = 33
original_labels    = ['abuse', 'adoration', 'annoyance', 'awkwardness', 'benefit', 'boredom', 'calmness', 'challenge', 'cheer', 'confusion', 'curiosity', 'desire', 'excitement', 'guilt', 'horror', 'humour', 'impressed', 'loss', 'nervousness', 'nostalgia', 'pain', 'relief', 'satisfaction', 'scold', 'shock', 'sympathy', 'threat']
train_count        = [31, 190, 1051, 880, 220, 78, 752, 214, 534, 486, 545, 180, 867, 216, 280, 153, 257, 351, 398, 65, 36, 173, 136, 94, 372, 209, 263]

EMOTIONS           = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

# DataLoader Hyperparamaters
BATCH_SIZE = 64

# Module 1 hyperparamaters(speaker_specific_emotion_sequence) : GRU n-n
input_size_1  = 7
hidden_size_1 = 10 
num_layers_1  = 2 
output_size_1 = 10


# Module 2 hyperparamaters(utterance_context) : Transformer Enc
input_size_2 = 768
n_head_2     = 4
dm_ff_2      = 2048
dp_2         = 0.2
num_layers_2 = 4 
act_fn_2     = 'relu'

# Module 3 hyperparamaters(speaker_context) : Transformer Enc
input_size_3 = 232
n_head_3     = 4
dm_ff_3      = 2048
dp_3         = 0.2
num_layers_3 = 4 
act_fn_3     = 'relu'

# Module 4 hyperparamaters(global_emotion_sequence) : GRU
input_size_4  = 7
hidden_size_4 = 10 
num_layers_4  = 2 
output_size_4 = 7

# Final Model Hyperparamerters:
fc1_out = 800
fc2_out = 800
fc3_out = 400
fc4_out = 100
fc5_out = len(original_labels)

learning_rate = 0.0001

Using device =>  cpu  torch  2.1.1+cu121


In [28]:
with open('./home/sgarc/SemEval-2023-Task-10/train_df.pkl', 'rb') as f:
    train_df = pickle.load(f)

,episode,speakers,emotions,utterances,triggers,sentence_embeddings,valence,valence_speaker
0,utterance_0,"[[False, False, False, False, False, False, Fa...","[[False, False, False, False, True, False, Fal...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 1.0, 0.0]","[0.0154345334, 0.1910427213, -0.1446124464, 0....","[0.14725, 0.10033333333333333, 0.0, 0.18277777...","[[0.14725, -1.0, 0.0, -1.0, 0.185], [-1.0, 0.1..."
1,utterance_1,"[[False, False, False, False, False, False, Fa...","[[False, False, False, False, True, False, Fal...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0.0137472358, 0.173657611, -0.1451763064, 0.1...","[0.14725, 0.10033333333333333, 0.0, 0.18277777...","[[0.14725, -1.0, 0.0, -1.0, 0.185, -1.0, 0.317..."
2,utterance_2,"[[False, False, False, False, False, False, Fa...","[[False, False, False, False, True, False, Fal...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0132562984, 0.1341271251, -0.1495521367, 0....","[0.14725, 0.10033333333333333, 0.0, 0.18277777...","[[0.14725, -1.0, 0.0, -1.0, 0.185, -1.0, 0.317..."
3,utterance_3,"[[False, False, False, False, False, False, Fa...","[[False, False, False, False, True, False, Fal...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0091593927, 0.1393967271, -0.1447667778, 0....","[0.14725, 0.10033333333333333, 0.0, 0.18277777...","[[0.14725, -1.0, 0.0, -1.0, 0.185, -1.0, 0.317..."
4,utterance_4,"[[False, False, False, False, False, False, Fa...","[[False, False, False, False, False, False, Tr...",[But then who? The waitress I went out with la...,"[0.0, 0.0, 1.0, 0.0]","[0.0270308554, 0.1875266433, -0.1426392645, 0....","[0.12127272727272728, 0.26925, 0.0, 0.10928571...","[[0.12127272727272728, -1.0, 0.0, -1.0], [-1.0..."
...,...,...,...,...,...,...,...,...
3995,utterance_3995,"[[False, False, False, False, False, False, Fa...","[[False, False, False, False, True, False, Fal...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0127896583, 0.1395556927, -0.1457487643, 0....","[0.0, 0.0, 0.0, 0.049, 0.24422222222222223, 0....","[[0.0, -1.0, -1.0, 0.049, -1.0, 0.052090909090..."
3996,utterance_3996,"[[False, False, False, False, False, False, Fa...","[[False, False, False, False, True, False, Fal...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0054899799, 0.1387219727, -0.1431471705, 0....","[0.0, 0.0, 0.0, 0.049, 0.24422222222222223, 0....","[[0.0, -1.0, -1.0, 0.049, -1.0, 0.052090909090..."
3997,utterance_3997,"[[False, False, False, False, False, False, Fa...","[[False, False, False, False, True, False, Fal...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0006412248, 0.1379950494, -0.1410688758, 0....","[0.0, 0.0, 0.0, 0.049, 0.24422222222222223, 0....","[[0.0, -1.0, -1.0, 0.049, -1.0, 0.052090909090..."
3998,utterance_3998,"[[False, False, False, False, False, False, Fa...","[[False, False, False, False, True, False, Fal...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0025664978, 0.1363260895, -0.1433342695, 0....","[0.0, 0.0, 0.0, 0.049, 0.24422222222222223, 0....","[[0.0, -1.0, -1.0, 0.049, -1.0, 0.052090909090..."


In [29]:
class SemEvalDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.len = len(self.data)
        print(list(train_df.columns))
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        dict_x = {}
        dict_x['speaker'] = torch.tensor(self.data['speakers'][index], dtype=torch.int)
        dict_x['emotion'] = torch.tensor(self.data['emotions'][index], dtype=torch.int)
        dict_x['sentence_embeddings'] = torch.tensor(self.data['sentence_embeddings'][index], dtype=torch.float64)
        dict_x['valence'] = torch.tensor(self.data['valence'][index])

        dict_y = {}
        dict_y['triggers'] =  torch.tensor(self.data['triggers'][index], dtype=torch.float32)

        return dict_x, dict_y

In [30]:
train_dataset = SemEvalDataset(train_df)

['episode', 'speakers', 'emotions', 'utterances', 'triggers', 'sentence_embeddings', 'valence', 'valence_speaker']


In [31]:
max(len(i) for i in train_df['speakers'])

24

In [32]:
class MELDCollate:
    def __init__(self, pad_value = 0):
        self.pad_value = pad_value
    def __call__(self, batch):
        speaker             = pad_sequence([item[0]['speaker'] for item in batch], batch_first = True)
        emotion             = pad_sequence([item[0]['emotion'] for item in batch], batch_first = True)
        sentence_embeddings = pad_sequence([item[0]['sentence_embeddings'] for item in batch], batch_first = True)
        valence             = pad_sequence([item[0]['valence'] for item in batch], batch_first = True)
        # print('\noriginal list : ',[item[0]['speaker'] for item in batch], '\n\npadded list : ', speaker)
        labels              = pad_sequence([item[1]['triggers'] for item in batch], batch_first = True)

        dict_x = { 'speaker': speaker, 'emotion':emotion,  'sentence_embeddings':sentence_embeddings, 'valence':valence}
        dict_y = {'labels': labels}

        return dict_x, dict_y

In [33]:
train_loader  = DataLoader(dataset = train_dataset, batch_size = 64, shuffle=True, collate_fn= MELDCollate())

In [34]:
desired_batch_index = 6
for i, batch in enumerate(train_loader):
    if i == desired_batch_index:
        # 'batch' contendrá el batch en el índice especificado
        print(f"Batch {i}:")
        print(batch)
        break

Batch 6:
({'speaker': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 

In [35]:
tesst = train_dataset.__getitem__(45)
print(len(tesst[0]['speaker'].unique(dim=0)))
print(tesst[0]['valence'])
print(tesst[0]['speaker'].unique(dim = 0, return_inverse=True ))
test = tesst[0]['speaker'].unique(dim = 0, return_inverse=True)[1]
[torch.where(test == i , tesst[0]['valence'], 0) for i in test.unique()]x

5
tensor([0.2188, 0.1796, 0.3178, 0.1020, 0.4790, 0.4790, 0.2708, 0.1875, 0.4185,
        0.2677], dtype=torch.float64)
(tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 2, 0, 4, 1, 1, 0, 3, 3, 1]))


[tensor([0.2188, 0.0000, 0.3178, 0.0000, 0.0000, 0.0000, 0.2708, 0.0000, 0.0000,
         0.0000], dtype=torch.float64),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.4790, 0.4790, 0.0000, 0.0000, 0.0000,
         0.2677], dtype=torch.float64),
 tensor([0.0000, 0.1796, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000], dtype=torch.float64),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1875, 0.4185,
         0.0000], dtype=torch.float64),
 tensor([0.0000, 0.0000, 0.0000, 0.1020, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000], dtype=torch.float64)]

In [36]:
class Module1GRU(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size, output_size):
        super(Module1GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.output_size = output_size
        # Since there are maximum of 8 speakers in a dialogue, so we decided to make 8 GRUs one for each speaker.
        self.gru_list= []
        for id in range(MAX_NO_OF_SPEAKERS):
            self.gru_list.append(nn.GRU(input_size, hidden_size, num_layers, batch_first = True))
        self.gru_modules = nn.ModuleList(self.gru_list)

    def valence_specific(valence, speaker):
        speaker = speaker.unique(dim = 0, return_inverse=True)[1]
        return [torch.where(speaker == i , valence, 0) for i in speaker.unique()]
    
    